In [2]:
from pathlib import Path

from tira.rest_api_client import Client
from tira.third_party_integrations import get_output_directory

if __name__ == "__main__":

    tira = Client()

    # loading train data
    text_train = tira.pd.inputs(
        "nlpbuw-fsu-sose-24", "authorship-verification-train-20240408-training"
    )
    targets_train = tira.pd.truths(
        "nlpbuw-fsu-sose-24", "authorship-verification-train-20240408-training"
    )
    # loading validation data (automatically replaced by test data when run on tira)
    text_validation = tira.pd.inputs(
        "nlpbuw-fsu-sose-24", "authorship-verification-validation-20240408-training"
    )
    targets_validation = tira.pd.truths(
        "nlpbuw-fsu-sose-24", "authorship-verification-validation-20240408-training"
    )

In [60]:
import pandas as pd

merged_train = pd.merge(text_train, targets_train, on='id')
merged_validation = pd.merge(text_validation, targets_validation, on='id')


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=8000)
x_train = tfidf_vectorizer.fit_transform(merged_train['text']).toarray()
x_validation = tfidf_vectorizer.fit_transform(merged_validation['text']).toarray()

y_train = merged_train['generated']
y_validation = merged_validation['generated']

In [62]:
# x_train_with_id = pd.DataFrame(x_train, columns=tfidf_vectorizer.get_feature_names_out())
# x_train_with_id['id'] = merged_train['id']
# y_train_with_id = pd.DataFrame({'generated': y_train, 'id': merged_train['id']})

# x_validation_with_id = pd.DataFrame(x_train, columns=tfidf_vectorizer.get_feature_names_out())
# x_validation_with_id['id'] = merged_validation['id']
# y_validation_with_id = pd.DataFrame({'generated': y_train, 'id': merged_validation['id']})

In [65]:
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(x_train, y_train)

# Make predictions on the test data
predictions = model.predict(x_validation)

# Evaluate the model
accuracy = model.score(x_validation, y_validation)
print("Accuracy:", accuracy)

Accuracy: 0.5


In [67]:
from sklearn.metrics import f1_score

f1 = f1_score(y_validation, predictions)
print(f1)

0.6666666666666666
